In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# TEST PUBLIC

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

In [24]:
data_public_catboost = pd.read_csv('/content/drive/MyDrive/DATATHON - 2/df_mixed_imputed_public_probas_catboost.csv')
data_public_lgbm = pd.read_csv('/content/drive/MyDrive/DATATHON - 2/df_mixed_imputed_public_probas.csv')

In [4]:
data_private_catboost

,Unnamed: 0,CHD_OR_MI,ID,Predicciones_catboos_public,Predicciones_proba_catboos_public
0,0,1.0,PID2022_100531,1.0,0.919020
1,1,1.0,PID2022_043184,1.0,0.982532
2,2,1.0,PID2022_071971,1.0,0.878206
3,3,1.0,PID2022_070778,1.0,0.991603
4,4,1.0,PID2022_082363,1.0,0.883105
...,...,...,...,...,...
43401,43401,0.0,PID2022_256399,1.0,0.733915
43402,43402,1.0,PID2022_326390,1.0,0.992920
43403,43403,0.0,PID2022_178405,1.0,0.864083
43404,43404,1.0,PID2022_220522,1.0,0.988278


In [5]:
data_public_lgbm

,Unnamed: 0,CHD_OR_MI,ID,Predicciones,Predicciones_proba
0,0,1.0,PID2022_100531,1,0.806681
1,1,1.0,PID2022_043184,1,0.978648
2,2,1.0,PID2022_071971,1,0.862523
3,3,1.0,PID2022_070778,1,0.984915
4,4,1.0,PID2022_082363,1,0.843071
...,...,...,...,...,...
43401,43401,0.0,PID2022_256399,1,0.657047
43402,43402,1.0,PID2022_326390,1,0.986876
43403,43403,0.0,PID2022_178405,1,0.796577
43404,43404,1.0,PID2022_220522,1,0.985622


In [7]:
proba_model1 = data_public_catboost['Predicciones_proba_catboos_public']
proba_model2 = data_public_lgbm['Predicciones_proba']

# Etiquetas reales
y_true = data_private_lgbm['CHD_OR_MI']

Aqui se estan generando los pesos para averiguar qué peso y bajo que umbral hace que el f1score sea mejor.

In [9]:
weights = np.linspace(0, 1, 101)  # 101 valores entre 0 y 1

best_f1 = 0
best_weight = 0
best_threshold = 0

# Iterar sobre diferentes pesos y umbrales
for weight in weights:
    # Combinación ponderada de probabilidades
    combined_proba = weight * proba_model1 + (1 - weight) * proba_model2

    # Probar diferentes umbrales para convertir probabilidades en predicciones
    for threshold in np.linspace(0, 1, 101):
        y_pred = (combined_proba >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)

        # Guardar el mejor F1-score y sus parámetros
        if f1 > best_f1:
            best_f1 = f1
            best_weight = weight
            best_threshold = threshold

print(f"Mejor F1-score: {best_f1:.4f}")
print(f"Mejor peso para el modelo 1: {best_weight:.2f}")
print(f"Mejor umbral: {best_threshold:.2f}")

Mejor F1-score: 0.9581
Mejor peso para el modelo 1: 0.79
Mejor umbral: 0.50


In [27]:
best_f1

0.9581323557785066

Se puede observar que el f1 score mejora.

In [17]:
# Combinación ponderada de probabilidades con el mejor peso
best_combined_proba = best_weight * proba_model1 + (1 - best_weight) * proba_model2

# Las probabilidades ya están calculadas
best_y_pred = (best_combined_proba >= best_threshold).astype(int)

# Si quieres las probabilidades combinadas que produjeron el mejor F1-score:
print(f"Mejores probabilidades: {best_y_pred}")

Mejores probabilidades: 0        1
1        1
2        1
3        1
4        1
        ..
43401    1
43402    1
43403    1
43404    1
43405    1
Length: 43406, dtype: int64


In [25]:
data_FINAL_public = pd.concat([data_public_lgbm[['ID']],best_y_pred],axis=1)
data_FINAL_public

,ID,0
0,PID2022_100531,1
1,PID2022_043184,1
2,PID2022_071971,1
3,PID2022_070778,1
4,PID2022_082363,1
...,...,...
43401,PID2022_256399,1
43402,PID2022_326390,1
43403,PID2022_178405,1
43404,PID2022_220522,1


# TEST PRIVATE


In [13]:
data_private_catboost = pd.read_csv('/content/drive/MyDrive/DATATHON - 2/df_mixed_imputed_private_probas_catboost.csv')
data_private_lgbm = pd.read_csv('/content/drive/MyDrive/DATATHON - 2/df_mixed_imputed_private_probas.csv')

In [15]:
data_private_lgbm

,Unnamed: 0,ID,Predicciones_private,Predicciones_proba_private
0,0,PID2022_152435,1,0.711385
1,1,PID2022_299594,1,0.978955
2,2,PID2022_065147,1,0.994449
3,3,PID2022_333651,1,0.979612
4,4,PID2022_317306,1,0.989775
...,...,...,...,...
43401,43401,PID2022_405631,1,0.978743
43402,43402,PID2022_264185,1,0.575913
43403,43403,PID2022_019640,1,0.786368
43404,43404,PID2022_073050,1,0.853941


In [19]:
proba_model1_private = data_private_catboost['Predicciones_proba_catboos_private']
proba_model2_private = data_private_lgbm['Predicciones_proba_private']


# Combinación ponderada de probabilidades con el mejor peso
best_combined_proba_PRIVATE = best_weight * proba_model1_private + (1 - best_weight) * proba_model2_private

# Las probabilidades ya están calculadas, pero si quieres convertirlas en predicciones:
best_y_pred_PRIVATE = (best_combined_proba_PRIVATE >= best_threshold).astype(int)

# Si quieres las probabilidades combinadas que produjeron el mejor F1-score:
print(f"Mejores probabilidades combinadas: {best_y_pred_PRIVATE}")

Mejores probabilidades combinadas: 0        1
1        1
2        1
3        1
4        1
        ..
43401    1
43402    1
43403    1
43404    1
43405    1
Length: 43406, dtype: int64


In [20]:
best_y_pred_PRIVATE

,0
0,1
1,1
2,1
3,1
4,1
...,...
43401,1
43402,1
43403,1
43404,1


In [22]:
data_FINAL = pd.concat([data_private_lgbm[['ID']],best_y_pred_PRIVATE],axis=1)
data_FINAL

,ID,0
0,PID2022_152435,1
1,PID2022_299594,1
2,PID2022_065147,1
3,PID2022_333651,1
4,PID2022_317306,1
...,...,...
43401,PID2022_405631,1
43402,PID2022_264185,1
43403,PID2022_019640,1
43404,PID2022_073050,1


In [26]:
base_final = pd.concat([data_FINAL_public,data_FINAL],axis=0)
print(base_final.shape)
base_final.to_csv('resultados.csv')

(86812, 2)
